In [1]:
# --- TEST DATA_LOADER ---

from src import data_loader as dl
from src import portfolio_processor as pp
from src import market_data_loader as mdl
import pandas as pd
import os
import sys
import json

filepath = r'data/U13271915_20250101_20251029.csv'
output_excel_path = r'data/report_output.xlsx'

pd.set_option('display.max_rows', 100)
pd.set_option('display.max_columns', 10)
pd.set_option('display.width', 150)

print(f"Loading report from: {filepath}...")
results = dl.load_ibkr_report(filepath)

if results:
    print("\nReport loaded successfully.")
    
    try:
        # Use pd.ExcelWriter to save both DataFrames to one file
        # on separate sheets.
        with pd.ExcelWriter(output_excel_path) as writer:
            results['initial_state'].to_excel(writer, sheet_name='initial_state', index=False)
            results['events'].to_excel(writer, sheet_name='events', index=False)
            results['financial_info'].to_excel(writer, sheet_name='financial_info', index=False)
            
        print(f"Successfully saved data to: {output_excel_path}")
        
    except ImportError:
        print("\n--- ERROR ---")
        print("To save to Excel, you need the 'openpyxl' library.")
        print("Please install it by running this in your terminal:")
        print("pip install openpyxl")
    except Exception as e:
        print(f"\nAn error occurred while saving to Excel: {e}")

    # --- 4. Print results to console as before ---
    print("\n" + "="*80 + "\n")
    print("--- 1. Initial State DataFrame ---")
    print(results['initial_state'])
    
    print("\n" + "="*80 + "\n")
    print("--- 2. Master Event Log ---")
    print(results['events'])

else:
    print("Data loading failed. No results to save.")

Loading report from: data/U13271915_20250101_20251029.csv...

Report loaded successfully.
Successfully saved data to: data/report_output.xlsx


--- 1. Initial State DataFrame ---
   symbol asset_category currency      quantity  value_native
0    ANFO          Stock      CHF      0.000000      0.000000
1    ASML          Stock      USD      2.000000   1386.160000
2   CHDVD          Stock      CHF     45.000000   7106.400000
3     DGE          Stock      GBP     56.000000   1421.000000
4    EMBC          Stock      CHF      0.000000      0.000000
5    FLIN          Stock      USD    192.000000   7276.800000
6    FONC          Stock      CHF      0.000000      0.000000
7      HO          Stock      EUR      0.000000      0.000000
8    IBKR          Stock      USD      3.036800    536.511456
9    IHYU          Stock      CHF      0.000000      0.000000
10   IUSC          Stock      CHF     86.000000   7833.740000
11    IVV          Stock      USD     12.000000   7064.160000
12    KGX      

In [ ]:
# ---

filepath = r'data/U13271915_20250101_20251029.csv'
output_excel_path = r'data/data_processing_output.xlsx'

# Make sure the 'data' directory exists
os.makedirs('data', exist_ok=True)

original_data = dl.load_ibkr_report(filepath)

# "results" will now be your big dictionary
results = pp.adjust_for_splits(original_data)

# --- Save to Excel ---
try:
    with pd.ExcelWriter(output_excel_path) as writer:
        # Save Original
        original_data['initial_state'].to_excel(writer, sheet_name='Initial_State_Orig', index=False)
        original_data['events'].to_excel(writer, sheet_name='Event_Log_Orig', index=False)
        
        # Save Adjusted and Debug files
        results['initial_state_adj'].to_excel(writer, sheet_name='Initial_State_Adj', index=False)
        results['events_adj'].to_excel(writer, sheet_name='Events_Log_Adj', index=False)
        results['adj_factors_wide'].to_excel(writer, sheet_name='Adj_Factors_Wide')
        results['adj_factors_long'].to_excel(writer, sheet_name='Adj_Factors_Long', index=False)
        results['events_with_factors'].to_excel(writer, sheet_name='Events_w_Factors', index=False)
        results['splits_found'].to_excel(writer, sheet_name='Splits_Found', index=False)
    print(f"Successfully saved all data to: {output_excel_path}")
except Exception as e:
    print(f"An error occurred while saving to Excel: {e}")


# --- Print to Console ---
print("--- Adjustment Factors (Wide) ---")
print(results['adj_factors_wide'])

print("\n--- Adjustment Factors (Long) ---")
print(results['adj_factors_long'])

print("\n--- Final Adjusted Events (Head) ---")
print(results['events_adj'].head())

Successfully saved all data to: data/data_processing_output.xlsx
--- Adjustment Factors (Wide) ---
            ANFO  ASML  CHDVDz  DGE  EMBC  ...  SPY5z  UNH  VGLT  VNAd  VTI
2025-01-01   1.0   1.0     1.0  1.0   1.0  ...    1.0  1.0   1.0   1.0  1.0
2025-01-02   1.0   1.0     1.0  1.0   1.0  ...    1.0  1.0   1.0   1.0  1.0
2025-01-03   1.0   1.0     1.0  1.0   1.0  ...    1.0  1.0   1.0   1.0  1.0
2025-01-04   1.0   1.0     1.0  1.0   1.0  ...    1.0  1.0   1.0   1.0  1.0
2025-01-05   1.0   1.0     1.0  1.0   1.0  ...    1.0  1.0   1.0   1.0  1.0
...          ...   ...     ...  ...   ...  ...    ...  ...   ...   ...  ...
2025-10-25   1.0   1.0     1.0  1.0   1.0  ...    1.0  1.0   1.0   1.0  1.0
2025-10-26   1.0   1.0     1.0  1.0   1.0  ...    1.0  1.0   1.0   1.0  1.0
2025-10-27   1.0   1.0     1.0  1.0   1.0  ...    1.0  1.0   1.0   1.0  1.0
2025-10-28   1.0   1.0     1.0  1.0   1.0  ...    1.0  1.0   1.0   1.0  1.0
2025-10-29   1.0   1.0     1.0  1.0   1.0  ...    1.0  1.0   1.0 

In [ ]:


# --- 1. Setup Environment ---
imports_ok = False
if 'src' not in sys.path:
    sys.path.append('src')

try:
    # Import your two custom modules
    import data_loader as dl
    import market_data_loader as mdl
    imports_ok = True
    
except ImportError as e:
    print(f"ImportError: {e}")
    print("Error: Could not import 'data_loader' or 'market_data_loader'.")
    print("\n==================== IMPORTANT ====================")
    print("If the error mentions 'yfinance' or 'openpyxl', run this in a new cell:")
    print("!pip install yfinance openpyxl")
    print("\nThen, **RESTART THE KERNEL** and run this cell again.")
    print("=================================================")

# --- 2. Run Test if Imports Succeeded ---
if imports_ok:
    
    FILE_PATH = r'data/U13271915_20250101_20251029.csv'
    EXCEL_OUTPUT_PATH = r'data/market_data_output.xlsx' # Define the output path
    
    print(f"Loading report from: {FILE_PATH}...")
    report_data = dl.load_ibkr_report(FILE_PATH)

    if report_data:
        print("Report loaded successfully.")
        print("Fetching market data (this may take a moment)...")
        
        # --- 3. Call the Function ---
        market_data_bundle = mdl.fetch_market_data(report_data)
        
        # --- 4. Display Results ---
        
        # First, print the debugging map
        print("\n" + "="*80)
        print("--- Ticker Info Map (Debugging) ---")
        print(json.dumps(market_data_bundle['ticker_info'], indent=2, ensure_ascii=False))
        print("="*80)

        # Second, print a summary of all data that was fetched
        print("\n--- Fetched Data Summary ---")
        for symbol, df in market_data_bundle['data'].items():
            info = market_data_bundle['ticker_info'].get(symbol, {})
            ticker = info.get('yfinance_ticker', 'N/A')
            status = info.get('status', 'N/A')
            
            print(f"  Symbol: {symbol} (Ticker: {ticker})")
            
            if not df.empty:
                print(f"  Date Range: {df.index.min().date()} to {df.index.max().date()}")
                print(f"  Rows: {len(df)}")
            else:
                print(f"  Status: {status}")
            print("-" * 20)
        
        # --- 5. Save to Excel ---
        print(f"\nSaving all fetched market data to: {EXCEL_OUTPUT_PATH}...")
        
        try:
            with pd.ExcelWriter(EXCEL_OUTPUT_PATH) as writer:
                # Loop through each symbol and its DataFrame in the 'data' dictionary
                for symbol, df in market_data_bundle['data'].items():
                    if not df.empty:
                        # Use the original IBKR symbol (e.g., "SPY5z") as the sheet name
                        # Excel limits sheet names to 31 chars, our symbols are fine.
                        df.to_excel(writer, sheet_name=symbol)
            
            print(f"Successfully saved data to {EXCEL_OUTPUT_PATH}")

        except ImportError:
            # This error happens if 'openpyxl' is not installed
            print("\n--- ERROR: 'openpyxl' is required to save to Excel ---")
            print("Please run this in a new cell:")
            print("!pip install openpyxl")
            print("Then re-run this script.")
        except Exception as e:
            print(f"\nAn error occurred while saving to Excel: {e}")
            
    else:
        print(f"Failed to load report from {FILE_PATH}.")
else:
    print("\nScript halted because critical modules could not be imported.")

Loading report from: data/U13271915_20250101_20251029.csv...
Report loaded successfully.
Fetching market data (this may take a moment)...
Fetching 27 tickers (Stocks & FX) from 2020-01-01 to 2025-10-30...
Tickers: ANFO.SW, ASML, CHDVD.SW, DGE.L, EMBC.SW, EURCHF=X, FLIN, FONC.SW, GBPCHF=X, HO.PA, IBKR, IGLB, IHYU.SW, IUSC.SW, IVV, KGX.F, MC.PA, MEUD.PA, MSFT, NESN.SW, RHM.F, SPY5.SW, UNH, USDCHF=X, VGLT, VNA.F, VTI
Creating 1.0 price history for base currency: CHF
Market data fetch complete.

--- Ticker Info Map (Debugging) ---
{
  "IHYUz": {
    "asset_type": "Stock",
    "original_symbol": "IHYUz",
    "yfinance_ticker": "IHYU.SW",
    "status": "OK"
  },
  "UNH": {
    "asset_type": "Stock",
    "original_symbol": "UNH",
    "yfinance_ticker": "UNH",
    "status": "OK"
  },
  "ASML": {
    "asset_type": "Stock",
    "original_symbol": "ASML",
    "yfinance_ticker": "ASML",
    "status": "OK"
  },
  "DGE": {
    "asset_type": "Stock",
    "original_symbol": "DGE",
    "yfinance_ticker

In [ ]:
# --- 1. Setup Environment ---
imports_ok = False
if 'src' not in sys.path:
    sys.path.append('src')

try:
    # Import all four of your custom modules
    import data_loader as dl
    import portfolio_processor as pp
    import market_data_loader as mdl
    import portfolio_reconstructor as pr # <-- NEW IMPORT
    
    imports_ok = True
    
except ImportError as e:
    print(f"ImportError: {e}")
    print("Error: Could not import one or more modules.")
    print("\n==================== IMPORTANT ====================")
    print("If the error mentions 'yfinance', 'openpyxl', or 'dateutil', run this in a new cell:")
    print("!pip install yfinance openpyxl python-dateutil")
    print("\nThen, **RESTART THE KERNEL** and run this cell again.")
    print("=================================================")

# --- 2. Run Full Pipeline if Imports Succeeded ---
if imports_ok:
    
    # --- Define File Paths ---
    FILE_PATH = r'data/U13271915_20250101_20251029.csv'
    EXCEL_OUTPUT_PATH = r'data/portfolio_reconstruction_output.xlsx'
    
    print(f"--- 1. Loading Report: {FILE_PATH} ---")
    original_report = dl.load_ibkr_report(FILE_PATH)

    if original_report:
        print("Report loaded successfully.")
        print(f"  Base Currency: {original_report['base_currency']}")
        print(f"  Report Period: {original_report['report_start_date']} to {original_report['report_end_date']}")
        
        # --- 2. Adjust for Splits ---
        print("\n--- 2. Adjusting for Splits ---")
        adjusted_report = pp.adjust_for_splits(original_report)
        print("Portfolio events adjusted for splits.")
        
        # --- 3. Fetch Market Data ---
        print("\n--- 3. Fetching Market Data (Stocks & FX) ---")
        print("This may take a moment...")
        market_data_bundle = mdl.fetch_market_data(original_report)
        print("Market data fetch complete.")
        
        # --- 4. Reconstruct Portfolio ---
        if market_data_bundle:
            print("\n--- 4. Reconstructing Daily Portfolio Time Series ---")
            portfolio_ts = pr.reconstruct_portfolio(
                adjusted_report, 
                market_data_bundle, 
                original_report
            )
            
            # --- 5. Save Results to Excel ---
            print(f"\n--- 5. Saving Results to Excel ---")
            print(f"Saving all {len(portfolio_ts)} asset time series to: {EXCEL_OUTPUT_PATH}")
            try:
                with pd.ExcelWriter(EXCEL_OUTPUT_PATH) as writer:
                    for symbol, df in portfolio_ts.items():
                        # Use the symbol as the sheet name
                        df.to_excel(writer, sheet_name=symbol)
                print(f"Successfully saved to {EXCEL_OUTPUT_PATH}")
            
            except ImportError:
                print("\n--- ERROR: 'openpyxl' is required to save to Excel ---")
                print("Please run this in a new cell: !pip install openpyxl")
            except Exception as e:
                print(f"\nAn error occurred while saving to Excel: {e}")

            # --- 6. Show Sample Output ---
            print("\n--- Sample Time Series for 'ASML' (first 5 days) ---")
            if 'ASML' in portfolio_ts:
                print(portfolio_ts['ASML'].head())
            
            print("\n--- Sample Time Series for 'USD' (first 5 days) ---")
            if 'USD' in portfolio_ts:
                print(portfolio_ts['USD'].head())

            print("\n--- Sample Time Series for 'CHF' (first 5 days) ---")
            if 'CHF' in portfolio_ts:
                print(portfolio_ts['CHF'].head())
        
        else:
            print("Market data bundle is empty. Halting.")
    else:
        print(f"Failed to load report from {FILE_PATH}.")
else:
    print("\nScript halted because critical modules could not be imported.")

--- 1. Loading Report: data/U13271915_20250101_20251029.csv ---
Report loaded successfully.
  Base Currency: CHF
  Report Period: 2025-01-01 to 2025-10-29

--- 2. Adjusting for Splits ---
Portfolio events adjusted for splits.

--- 3. Fetching Market Data (Stocks & FX) ---
This may take a moment...
Fetching 27 tickers (Stocks & FX) from 2020-01-01 to 2025-10-30...
Tickers: ANFO.SW, ASML, CHDVD.SW, DGE.L, EMBC.SW, EURCHF=X, FLIN, FONC.SW, GBPCHF=X, HO.PA, IBKR, IGLB, IHYU.SW, IUSC.SW, IVV, KGX.F, MC.PA, MEUD.PA, MSFT, NESN.SW, RHM.F, SPY5.SW, UNH, USDCHF=X, VGLT, VNA.F, VTI
Creating 1.0 price history for base currency: CHF
Market data fetch complete.
Market data fetch complete.

--- 4. Reconstructing Daily Portfolio Time Series ---
Successfully reconstructed portfolio time series for 27 assets.

--- 5. Saving Results to Excel ---
Saving all 27 asset time series to: data/portfolio_reconstruction_output.xlsx
Successfully saved to data/portfolio_reconstruction_output.xlsx

--- Sample Time S